<a href="https://colab.research.google.com/github/Rosilanny/Data_Science/blob/main/Case_T%C3%A9cnico_RAIA_Fellowship_2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Análise de sentimentos por meio de comentários**

##**Contexto**

O conjunto de dados que utilizaremos para esta solução consiste em uma coluna de dados de  comentários curtos e informais feitos por usuários sobre um serviço ou aplicativo. Aqui consideraremos como se fosse comentários de um ecommerce sobre um determinado produto.  

Os textos apresentam variações típicas da linguagem cotidiana, como abreviações, baixa contextualização, ambiguidades semânticas e vocabulário diverso.

Um aspecto importante dessa base de dados é  que não há rótulos de sentimento fornecidos. Dessa forma, não existe uma “verdade” prévia indicando se cada comentário é positivo, neutro ou negativo. Essa limitação impõe restrições importantes sobre as abordagens possíveis, especialmente no uso de métodos supervisionados tradicionais.

Dado esse contexto, o objetivo não é maximizar métricas de classificação, mas sim desenvolver uma solução tecnicamente adequada, interpretável e coerente com as características do dado, respeitando as restrições do problema.

Uma das aplicações dessa solução seria apoiar o time de marketing de produto a entender rapidamente a percepção dos usuários sobre o seu produto no ecommerce, separando comentários positivos, neutros e negativos.


 <p align=center>
<img src="https://img.freepik.com/vetores-premium/analise-de-sentimentos-diferentes-opinioes-varios-comentarios-de-clientes-boas-emocoes-neutras-e-ruins-pessoas-comunicacao-positiva-e-negativa-discussao-e-conflito-conceito-de-desenho-animado-vetor_176411-9200.jpg" width="40%"></p>


##**Estratégia geral da solução**

A solução foi pensada em duas camadas complementares:

1. Heurística baseada em score
Utilizada como solução principal, por ser interpretável, estável em conjuntos pequenos e adequada à ausência de rótulos. Essa heurística gera pseudo-rótulos (silver labels) a partir de regras linguísticas explícitas.

2. Modelo TF-IDF + Regressão Logística
Utilizado como confirmação/comparação, treinado para reproduzir os rótulos gerados pela heurística. As métricas associadas a esse modelo medem concordância entre métodos, e não erro em relação ao sentimento real dos usuários.

##**Importação e leitura dos dados**

In [ ]:
import re
import unicodedata
import pandas as pd

arquivo = "comentarios.txt"  # ajuste se necessário

with open(arquivo, "r", encoding="utf-8") as f:
    linhas = f.readlines()

comentarios = []
for linha in linhas:
    linha = linha.strip()
    if not linha:
        continue
    # formato esperado: "texto",
    m = re.search(r'"(.*)"', linha)
    if m:
        comentarios.append(m.group(1))
    else:
        comentarios.append(linha)

print("Total de comentários:", len(comentarios))
comentarios[:10]

Total de comentários: 50


['muito bom, gostei bastante',
 'funciona bem',
 'mais ou menos',
 'nao gostei',
 'app excelente']

##**Pré-processamento**

Processo de normalização do texto para reduzir ruido e ganharam uma análise mais limpa e assertiva. O objetivo central desse pré-processamento é reduzir a dispersão do vocabulário (ótimo/otimo/ÓTIMO viram o mesmo token)

In [ ]:
def remover_acentos(texto: str) -> str:
    texto = unicodedata.normalize("NFKD", texto)
    return "".join([c for c in texto if not unicodedata.combining(c)])

def normalizar_texto(texto: str) -> str:
    texto = texto.lower().strip()
    texto = remover_acentos(texto)
    texto = re.sub(r"[^a-z0-9\s]", " ", texto)   # removendo pontuação
    texto = re.sub(r"\s+", " ", texto).strip()   # removendo espaços extras
    return texto

comentarios_norm = [normalizar_texto(c) for c in comentarios]
list(zip(comentarios[:3], comentarios_norm[:3])) #visualizando a parte original e a parte normalizada


[('muito bom, gostei bastante', 'muito bom gostei bastante'),
 ('funciona bem', 'funciona bem'),
 ('mais ou menos', 'mais ou menos')]

##**Definindo o baseline :** Regras para rótulo inicial
Segue abaixo um baseline  norte para a classificação onde definiremos regras claras para o dois polos de classificação: negativo e positivo.

Utilizaremos essa abordadgem por dois fatores principais:

1.   Temos poucos dados (50 linhas)
2.   O conjunto de dados  não possui rótulos, ou seja, não temos a resposta correta do sentimento/classificação associada a cada dado (comentário).

Diante disso, não é possível aplicar diretamente métodos supervisionados de aprendizado de máquina sem a criação artificial de rótulos, o que poderia introduzir viés. Por esse motivo, foi adotada uma abordagem baseada em regras e heurísticas que chamaremos de baseline.

A heurística utiliza listas de palavras indicativas de polaridade e um score acumulado, evitando decisões frágeis baseadas em uma única ocorrência. Também é tratado um padrão simples de negação, comum em linguagem natural.

In [ ]:
palavras_positivas = [
    "bom","bem","boa","otimo","otima","excelente","maravilhoso","perfeito",
    "adorei","gostei","amei","recomendo","show","top","funciona","cumpre","promete","resolve"
]

palavras_negativas = [
    "ruim","horrivel","pessimo","pessima","odiei","deixa",
    "lento","demora","trava","travando","bug","erro","falha","problema",
    "decepcionante"
]

pistas_neutras = ["ok","normal","razoavel","mais ou menos","nada demais","bom mas","ruim mas","inho","podia"]

negacoes = ["nao"]  # após normalização

def classificar_sentimento_score(texto_normalizado: str) -> str:
    tokens = texto_normalizado.split()
    score = 0

    for i, palavra in enumerate(tokens):
        palavra_anterior = tokens[i-1] if i > 0 else ""

        # positivo
        if palavra in palavras_positivas:
            if palavra_anterior in negacoes:   # "nao gostei"
                score -= 1
            else:
                score += 1

        # negativo
        if palavra in palavras_negativas:
            if palavra_anterior in negacoes:   # "nao ruim"
                score += 1
            else:
                score -= 1

    # neutro (sem evidência ou ambíguo)
    if score > 0:
        return "positivo"
    elif score < 0:
        return "negativo"
    else:
        return "neutro"


In [ ]:
df_resultado = pd.DataFrame({
    "comentario": comentarios,
    "comentario_norm": comentarios_norm
})

df_resultado["y_baseline"] = df_resultado["comentario_norm"].apply(classificar_sentimento_score)

df_resultado.head(25)


,comentario,comentario_norm,y_baseline
0,"muito bom, gostei bastante",muito bom gostei bastante,positivo
1,funciona bem,funciona bem,positivo
2,mais ou menos,mais ou menos,neutro
3,nao gostei,nao gostei,negativo
4,app excelente,app excelente,positivo
5,demora pra carregar,demora pra carregar,negativo
6,"ok, nada demais",ok nada demais,neutro
7,odiei a experiencia,odiei a experiencia,negativo
8,"bom, mas podia melhorar",bom mas podia melhorar,positivo
9,pessimo atendimento,pessimo atendimento,negativo


In [ ]:
#Distribuição do baseline
df_resumo = (
    df_resultado["y_baseline"]
    .value_counts()
    .to_frame(name="quantidade")
)

df_resumo["percentual(%)"] = df_resumo["quantidade"] / df_resumo["quantidade"].sum() * 100

df_resumo

,quantidade,percentual(%)
y_baseline,,
positivo,18,36.0
neutro,18,36.0
negativo,14,28.0


Ao aplicarmos o modelo heurístico observamos a predominância da classe neutro junto com a classe positiva.

##**Avaliação da classificação dos sentimentos por TF-IDF + Regressão Logística**

O modelo TF-IDF + Regressão Logística é utilizado para avaliar se um classificador supervisionado consegue reproduzir de forma consistente os padrões definidos pela heurística.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

X = df_resultado["comentario_norm"]
y = df_resultado["y_baseline"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.30,
    random_state=42,
    stratify=y
)

tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=1)
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

modelo = LogisticRegression(max_iter=2000)
modelo.fit(X_train_vec, y_train)

y_pred = modelo.predict(X_test_vec)

df_predicoes = pd.DataFrame({
    "comentario": X_test,
    "y_real": y_test.values,
    "y_pred": y_pred
})

df_predicoes.head(10)

,comentario,y_real,y_pred
43,decepcionante,negativo,positivo
17,horrivel nao recomendo,negativo,negativo
6,ok nada demais,neutro,positivo
21,funciona quando quer,positivo,positivo
38,recomendo,positivo,positivo
39,mais lento que o esperado,negativo,neutro
47,excelente parabens,positivo,positivo
28,app ok,neutro,positivo
48,nao vale a pena,neutro,negativo
40,boa experiencia,positivo,negativo


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification report:\n")
print(classification_report(y_test, y_pred, digits=3))

cm = confusion_matrix(y_test, y_pred, labels=["negativo","neutro","positivo"])
cm_df = pd.DataFrame(cm, index=["true_neg","true_neu","true_pos"], columns=["pred_neg","pred_neu","pred_pos"])
cm_df

Accuracy: 0.3333333333333333

Classification report:

              precision    recall  f1-score   support

    negativo      0.250     0.250     0.250         4
      neutro      0.000     0.000     0.000         6
    positivo      0.400     0.800     0.533         5

    accuracy                          0.333        15
   macro avg      0.217     0.350     0.261        15
weighted avg      0.200     0.333     0.244        15



,pred_neg,pred_neu,pred_pos
true_neg,1,1,2
true_neu,2,0,4
true_pos,1,0,4


**A accuracy **de aproximadamente 33% indica que o modelo TF-IDF concorda com a heurística em um nível próximo ao aleatório para três classes a cada 15, o que é esperado dado o conjunto reduzido, a ausência de rótulos reais e a presença de ambiguidade.

**A análise de precision e recall** mostra um viés claro para a classe positivo, com recall elevado (0.80), enquanto as classes negativo e, principalmente, neutro apresentam baixo desempenho.

**O macro F1-score **(0.26) é o indicador global mais informativo neste cenário, pois penaliza igualmente o fracasso na classe neutra.

**A matriz de confusão** confirma que a maioria dos erros ocorre pela classificação de comentários neutros ou negativos como positivos. Esses resultados reforçam que o TF-IDF atua adequadamente como método de comparação, enquanto a heurística baseada em score é mais apropriada como solução principal neste contexto.

In [ ]:
#Análise dos dados onde o TF concordou com o  baseline (Heuristica)
df_test = pd.DataFrame({
    "comentario_norm": X_test.values,
    "y_true_heur": y_test.values,   # "verdade" aqui = baseline (Heuristica)
    "y_pred_tfidf": y_pred
})

df_erros = df_test[df_test["y_true_heur"] == df_test["y_pred_tfidf"]].copy()
df_erros.head(50)

,comentario_norm,y_true_heur,y_pred_tfidf
1,horrivel nao recomendo,negativo,negativo
3,funciona quando quer,positivo,positivo
4,recomendo,positivo,positivo
6,excelente parabens,positivo,positivo
14,show de bola,positivo,positivo


In [ ]:
#Análise dos erros onde o TF discordou do baseline (Heuristica)
df_test = pd.DataFrame({
    "comentario_norm": X_test.values,
    "y_true_heur": y_test.values,   # "verdade" aqui = heurística (pseudo-rótulo)
    "y_pred_tfidf": y_pred
})

df_erros = df_test[df_test["y_true_heur"] != df_test["y_pred_tfidf"]].copy()
df_erros.head(50)

,comentario_norm,y_true_heur,y_pred_tfidf
0,decepcionante,negativo,positivo
2,ok nada demais,neutro,positivo
5,mais lento que o esperado,negativo,neutro
7,app ok,neutro,positivo
8,nao vale a pena,neutro,negativo
9,boa experiencia,positivo,negativo
10,experiencia razoavel,neutro,negativo
11,interface confusa,neutro,positivo
12,legalzinho,neutro,positivo
13,pessimo atendimento,negativo,positivo


In [ ]:
# Quantificação dos erros por par (classe verdadeira heurística -> predição TF-IDF)
erros_por_tipo = (
    df_erros
    .groupby(["y_true_heur","y_pred_tfidf"])
    .size()
    .sort_values(ascending=False)
)

erros_por_tipo

y_true_heur  y_pred_tfidf
neutro       positivo        4
             negativo        2
negativo     positivo        2
             neutro          1
positivo     negativo        1
dtype: int64

Como o conjunto não possui rótulos reais, dessa forma os “erros” mensurados refere-se a discordâncias entre a heurística (baseline interpretável) e
 o modelo TF-IDF + Regressão (supervisionado nos pseudo-rótulos).

Principais causas típicas de discordância:

1) **Vocabulário fora das listas**: a heurística não captura palavras novas; o TF-IDF pode capturar por coocorrência.
2) **Negação e contexto curto**: “não gostei” / “não é ruim” — se o TF-IDF não viu exemplos suficientes no treino, pode errar.
3) **Ambiguidade**: frases curtas como “até que vai” ou “quebra um galho” podem ser neutras/positivas dependendo da interpretação.
4) **Polaridade fraca**: quando há poucas evidências, a heurística tende a neutro; o TF-IDF pode “forçar” uma classe.


##**Análise do NPS**

O NPS tradicional usa notas 0–10 (promotores, neutros, detratores).  
Considerando que os dados que temos são de um determinado produto dessa froma podemos avaliá-lo como um produto promotor, neutro ou detrator para o portfólio de produtos. Aqui não será calculado um é NPS real, mas uma estimativa exploratória baseada em texto para classificarmos o produto.

Aqui calculamos um **proxy** a partir de sentimento:
- positivo ≈ promotor
- neutro ≈ neutro
- negativo ≈ detrator


In [ ]:
# NPS proxy usando as classes da heurística (sobre o dataset inteiro)
total = len(df_resultado)
pos = (df_resultado["y_baseline"] == "positivo").sum()
neg = (df_resultado["y_baseline"] == "negativo").sum()

pct_pos = pos / total
pct_neg = neg / total

nps_proxy = (pct_pos - pct_neg) * 100

print(f"Positivos: {pos}/{total} ({pct_pos:.1%})")
print(f"Negativos: {neg}/{total} ({pct_neg:.1%})")
print(f"NPS proxy (baseado em sentimento): {nps_proxy:.1f}")

Positivos: 18/50 (36.0%)
Negativos: 14/50 (28.0%)
NPS proxy (baseado em sentimento): 8.0


In [ ]:
# NPS proxy usando as classes do modelo TF-IDF + Regressão Logística (sobre o dataset inteiro)


# Previsões do modelo TF-IDF para todos os comentários
df_resultado_final=df_resultado.copy()
X_all_vec = tfidf.transform(df_resultado_final["comentario_norm"])
df_resultado_final["y_modelo"] = modelo.predict(X_all_vec)
df_resultado_final.head(10)

,comentario,comentario_norm,y_baseline,y_modelo
0,"muito bom, gostei bastante",muito bom gostei bastante,positivo,positivo
1,funciona bem,funciona bem,positivo,positivo
2,mais ou menos,mais ou menos,neutro,neutro
3,nao gostei,nao gostei,negativo,negativo
4,app excelente,app excelente,positivo,positivo
5,demora pra carregar,demora pra carregar,negativo,negativo
6,"ok, nada demais",ok nada demais,neutro,positivo
7,odiei a experiencia,odiei a experiencia,negativo,negativo
8,"bom, mas podia melhorar",bom mas podia melhorar,positivo,positivo
9,pessimo atendimento,pessimo atendimento,negativo,positivo


In [ ]:
#Distribuição do resultado do baseline
df_resultado_final_baseline = (
   df_resultado_final["y_baseline"]
    .value_counts()
    .to_frame(name="quantidade")
)

df_resultado_final_baseline["percentual(%)"] = df_resultado_final_baseline["quantidade"] / df_resultado_final_baseline["quantidade"].sum() * 100

df_resultado_final_baseline

,quantidade,percentual(%)
y_baseline,,
positivo,18,36.0
neutro,18,36.0
negativo,14,28.0


In [ ]:
#Distribuição do resultado do modelo TF
df_resultado_final_modelo = (
   df_resultado_final["y_modelo"]
    .value_counts()
    .to_frame(name="quantidade")
)

df_resultado_final_modelo["percentual(%)"] = df_resultado_final_modelo["quantidade"] / df_resultado_final_modelo["quantidade"].sum() * 100

df_resultado_final_modelo

,quantidade,percentual(%)
y_modelo,,
positivo,23,46.0
negativo,14,28.0
neutro,13,26.0


In [ ]:
total = len(df_resultado_final)
pos_modelo = (df_resultado_final["y_modelo"] == "positivo").sum()
neg_modelo = (df_resultado_final["y_modelo"] == "negativo").sum()

pct_pos_modelo = pos_modelo / total
pct_neg_modelo = neg_modelo / total

nps_proxy_modelo = (pct_pos_modelo - pct_neg_modelo) * 100

print(f"Positivos: {pos_modelo}/{total} ({pct_pos_modelo:.1%})")
print(f"Negativos: {neg_modelo}/{total} ({pct_neg_modelo:.1%})")
print(f"NPS proxy do modelo (baseado em sentimento): {nps_proxy_modelo:.1f}")

Positivos: 23/50 (46.0%)
Negativos: 14/50 (28.0%)
NPS proxy do modelo (baseado em sentimento): 18.0


In [ ]:
print(f"NPS proxy (Baseline-Heurística): {nps_proxy:.1f}")
print(f"NPS proxy (Modelo TF-IDF): {nps_proxy_modelo:.1f}")

NPS proxy (Baseline-Heurística): 8.0
NPS proxy (Modelo TF-IDF): 18.0


A escala abaixo representa a visão final  da percepção do produto

* < 0	percepção negativa
* 0 a 30	percepção razoável
* 30 a 70	percepção muito boa
* '> 70 percepção excelente

Dessa forma temos que tanto considerando o NPS calculado pelo método heuristico como pelo médio TF-IDF o produto em questão teria uma percepção considerada razoável o que implica que consiste em um produto que precisa de um ponto de atenção para melhor entrega de qualidade ao cliente.

##**Sugestões de próximos passos**

1. Expandir o conjunto rotulado manualmente e assim termos uma coluna de rótulo dos sentimentos real dos usuários;

2. Ajustar hiperparâmetros do modelo TF-IDF + Regressão logística.

3. Explorar validação cruzada pois em conjuntos pequenos, como o utilizado neste estudo, a performance do modelo pode variar significativamente dependendo da amostra selecionada para teste;

4. Refinar heurística com expressões multi-palavra.

5. Separar classificação binária (positivo vs negativo) e tratar neutro posteriormente com o intuito de reduzir confusões e melhorar a qualidade tanto do modelo heurístico quanto do modelo supervisionado.